In [105]:
import collections

import datasets

from stanza.models.constituency.tree_reader import read_trees
from stanza.models.constituency.parse_tree import Tree

In [2]:
conll_data = datasets.load_dataset("coref-data/conll2012_indiscrim", "english_v4")

Generating train split:   0%|          | 0/2802 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/343 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/348 [00:00<?, ? examples/s]

In [100]:
def get_all_noun_phrases(node: Tree):
    if node.label == "NP" or node.label == "NML":
        yield node
    if not node.is_leaf():
        for child in node.children:
            yield from get_all_noun_phrases(child)


def get_all_noun_phrase_indices(tree):
    # map words to indices and get all noun phrases
    str_indices = map(str, range(len(tree)))
    index_tree = tree.replace_words(str_indices)
    return get_all_noun_phrases(index_tree)


def get_np_mentions(sent_i, sentence):
    tokens = sentence["tokens"]
    parse_tree = sentence["misc"]["parse_tree"]

    if not parse_tree:
        return

    trees = read_trees(parse_tree)
    assert len(trees) == 1
    tree: Tree = trees[0]

    assert len(tree) == len(tokens)

    for np_node in get_all_noun_phrase_indices(tree):
        leaves = np_node.leaf_labels()
        yield [sent_i, int(leaves[0]), int(leaves[-1])]


def add_singletons(example):
    sentences = example["sentences"]
    coref_chains = example["coref_chains"]

    singleton_mentions = []
    for sent_i, sentence in enumerate(sentences):
        for m in get_np_mentions(sent_i, sentence):
            singleton_mentions.append(m)

    return {"singleton_mentions": singleton_mentions}



In [101]:
singleton_data = conll_data.map(add_singletons)

Map:   0%|          | 0/2802 [00:00<?, ? examples/s]

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

Map:   0%|          | 0/348 [00:00<?, ? examples/s]

In [107]:
# calculate precision and recall
for split in ["train", "validation", "test"]:
    recalled = 0
    total_coref = 0
    total_singleton = 0

    for example in singleton_data[split]:
        coref_chains = example["coref_chains"]
        singleton_mentions = example["singleton_mentions"]

        coref_mentions = set()
        for c in coref_chains:
            for m in c:
                coref_mentions.add(tuple(m))

        singleton_mentions = set([tuple(m) for m in singleton_mentions])

        total_coref += len(coref_mentions)
        total_singleton += len(singleton_mentions)

        recalled += len(singleton_mentions.intersection(coref_mentions))
        
    print(split)
    print(recalled, total_coref, total_singleton, float(recalled) / total_coref, float(recalled) / total_singleton)
    # what percent of coref are in singleton? what percent of singleton are in coref?

        

train
135246 155558 417580 0.8694249090371438 0.3238804540447339
validation
16427 19155 51924 0.8575828765335421 0.31636622756336186
test
16850 19764 53633 0.8525602104837078 0.31417224470009136


In [ ]:
# calculate singleton mentions using parse

# All noun phrases with distinct headwords are extracted from previously treebanked data
# Whenever head-sharing NPs are nested, the largest logical span is used in co-reference (see 2.4.1).
## Head-sharing NPs are two (or more) extracted entities, the shorter one(s) contained within the span of the longer,
## sharing the same content word as their headword.
## In such cases, the longest logical span should be used in co-reference with other mentions. 

# Possessive proper nouns (Fred's) are extracted from the treebanked data;
# however, possessive pronouns (his) must be manually extracted by the annotator and added to the list of mentions

# Proper noun PreMods can be co-referenced to existing noun phrases and/or other proper PreMods,
# and should be manually extracted by the annotator and added to the list of mentions.
# Non-proper and adjectival premodifiers are not eligible for co-reference (see 2.3).
## Premodifiers must be proper nouns
## Pre-modifying dates and monetary amounts are also eligible for co-reference
## Acronymic premodifiers should be co-referenced unless they refer to nationality

# Only the single-word head of the verb phrase is included in the span,
# even in cases where the entire verb phrase is the logical co-referent.

# Appositives: only the whole span is linked for IDENT

# Partitives: all and both can corefer in nesting

# in, at, to, from cannot be metonyms

